### 기본적인 RAG 의 구조

문서 로딩 ➡️ 청크 스플릿 ➡️ 임베딩 ➡️ 벡터스토어 ➡️ <u>**리트리버**</u>로 유사문장 파악 ➡️ 출력

- <u>주요 필요 사항</u>
    - **Multi-query** : RAG 시스템은 프롬프트 엔지니어링 없이도 좋은 결과를 내야함
        - 사용자의 질문을 여러개의 유사 질문으로 재 생성
    - **Parent-Document** : 참고 문서의 앞뒤 문맥을 잘 파악해 내야함
        - 청크를 포함하고 있는 페이지 documents를 불러온다.
        - 즉 유사문서의 부모(parent) 문서를 참조
    - **Self-Querying** : 쿼리 (시멘틱 이 아닌) 가 필요하다.
        - 문서 데이터 기반으로 필터링 생성 (쿼리 생성)
            - 평점이 9점 이상인 df 영화 추천
            - rating > 9, genre == "sf"
            - 스티븐 스필버그의 쥬라기 공원이 있습니다.
            - 엑셀과, csv 등의 테이블 데이터에 유리하다
    - **Time-weighted** : 최신 자료 위주로 참고하게 끔.

In [1]:
# !pip install -q langchain pypdf sentence_transformers chromadb openai

### Multi-query Retriever

In [2]:
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# loader
urls = "https://alltommysworks.com/gan/"
loader = WebBaseLoader(urls)
docs = loader.load()

# splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

# VectorDB
model_name = "jhgan/ko-sbert-nli" # 임베딩 이용
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceEmbeddings(model_name=model_name,
                                   encode_kwargs=encode_kwargs)
vector_db = Chroma.from_documents(splits, embeddings)

USER_AGENT environment variable not set, consider setting it to identify your requests.
/tmp/ipykernel_2727/3147827098.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name,
2024-12-05 16:30:42.052996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733383842.070148    2727 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733383842.075468    2727 cuda_bla

In [3]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
import json

with open("api_keys.json") as f:
    OPENAI_API_KEY = json.load(f)["openai"]

question = "gan 이란 무엇인가요? (한글로)"

llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
mq_retriever = MultiQueryRetriever.from_llm(
    retriever=vector_db.as_retriever(), llm=llm
)

In [4]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [5]:
find_docs = mq_retriever.get_relevant_documents(question)
print(len(find_docs))

/tmp/ipykernel_2727/1720634346.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  find_docs = mq_retriever.get_relevant_documents(question)
INFO:langchain.retrievers.multi_query:Generated queries: ['1. gan은 무엇을 의미하나요?', '2. gan이란 무슨 뜻인가요?', '3. gan은 어떤 것을 가리키는지 알려주세요.']


5


### Parent_document Retriever

In [6]:
from langchain.retrievers import ParentDocumentRetriever

In [7]:
from langchain.storage import InMemoryStore 
                            # dict 형태로 저장 parent와 child 가 엮여있어 저장됨
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
loaders = [
    PyPDFLoader("docs/doc_1.pdf"),
    PyPDFLoader("docs/portfolio.pdf"),
    PyPDFLoader("docs/resume.pdf"),
]

In [9]:
docs = []
for loader in loaders:
    docs.extend(loader.load_and_split())

In [10]:
model_name, encode_kwargs, embeddings

('jhgan/ko-sbert-nli',
 {'normalize_embeddings': True},
 HuggingFaceEmbeddings(client=SentenceTransformer(
   (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
 ), model_name='jhgan/ko-sbert-nli', cache_folder=None, model_kwargs={}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False))

In [11]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size = 250,
                                                chunk_overlap = 0)
vectorstore = Chroma(
    collection_name="full_docs",
    embedding_function=embeddings,
)
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore= vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

/tmp/ipykernel_2727/1083163943.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [12]:
retriever.add_documents(docs, ids=None)

In [13]:
sub_docs = vectorstore.similarity_search("사용한 주요 성과 지표")
# 자식 doc
print("자식 docs 의 토큰수")
for doc in sub_docs:
    print(len(doc.page_content))

자식 docs 의 토큰수
179
232
249
124


In [14]:
# 부모 docs
print("부모 docs 의 토큰수")
par_docs = retriever.get_relevant_documents("사용한 주요 성과 지표")
for r in par_docs:
    print(len(r.page_content))

부모 docs 의 토큰수
415
394
1463


In [15]:
print('First doc from sub_docs:',)
print('글길이:',len(sub_docs[0].page_content),"\n")
print(sub_docs[0].page_content)
print("-"*100)
print('First doc from sub_docs:',)
print('글길이:',len(par_docs[0].page_content),"\n")
print(par_docs[0].page_content)


First doc from sub_docs:
글길이: 179 

- ICT 인증 기관에 따라 평가지표 설정
MAPs, BLEUs, METEOR, ROUGEs 등
- 성능 분석 및 비교하여 모델 선택
- 훈련 결과 시각화
시스템 설계 및 구축
- Docker 컨테이너 구축(WSL2 우분투 이용)
- Streamlit 및 FastAPI 구축
- Log 설계하여 성능지표, 모델 결과 추적
----------------------------------------------------------------------------------------------------
First doc from sub_docs:
글길이: 415 

SNS 사진 분석 댓글 및 피드백 생성 모델
기여한 내용
5
전처리 과정
- 기존 어노테이션을 CoCo 데이터 형식으로 변환
- YOLO 형식에 맞게 레이블 및 이미지를 재구성
- 언어 모델 지식 증류 기법을 적용하여 데이터셋 생성 
모델 설계 및 Fitting
- 언어 모델 선정 및 파인튜닝
- 객체 인식 모델 선정 및 파인 튜닝
- 각 모델별 하이퍼 파라미터 튜닝  
- 모델 구조도 작성(draw.io)
모델 평가
- ICT 인증 기관에 따라 평가지표 설정
MAPs, BLEUs, METEOR, ROUGEs 등
- 성능 분석 및 비교하여 모델 선택
- 훈련 결과 시각화
시스템 설계 및 구축
- Docker 컨테이너 구축(WSL2 우분투 이용)
- Streamlit 및 FastAPI 구축
- Log 설계하여 성능지표, 모델 결과 추적


- 페이지 청크가 길때 : child 스플리터 뿐 아니라, parent 스플리터도 청크를 정해주어 분해한다.

In [16]:
# Splitter 설정
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# 임베딩 함수 설정
embedding_function = embeddings  # embeddings가 제대로 정의되었는지 확인하세요.

# VectorStore 설정
vectorstore = Chroma(
    collection_name="full_docs",
    embedding_function=embedding_function,
)

# InMemoryStore 설정
store = InMemoryStore()

# ParentDocumentRetriever 설정
retriever_double_splitter = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [17]:
retriever_double_splitter.add_documents(docs)

In [18]:
len(list(store.yield_keys()))

106

In [19]:
sub_docs = vectorstore.similarity_search("성과 지표")
# 자식 doc
print("자식 docs 의 토큰수")
for doc in sub_docs:
    print(len(doc.page_content))

자식 docs 의 토큰수
50
86
179
25


In [20]:
# 부모 docs
print("부모 docs 의 토큰수")
par_docs = retriever_double_splitter.get_relevant_documents("성과 지표")
for r in par_docs:
    print(len(r.page_content))

부모 docs 의 토큰수
150
197
197


In [21]:
print('First doc from sub_docs:',)
print('글길이:',len(sub_docs[0].page_content),"\n")
print(sub_docs[0].page_content)
print("-"*100)
print('First doc from sub_docs:',)
print('글길이:',len(par_docs[0].page_content),"\n")
print(par_docs[0].page_content)


First doc from sub_docs:
글길이: 50 

Precision, ROC-AUC, F1-Score등다양한성능지표를활용하여모델유효성평가 .
----------------------------------------------------------------------------------------------------
First doc from sub_docs:
글길이: 150 

다양한샘플링기법(ROS, RUS, SMOTE, ENN, SMOTEENN등 ) 을통해데이터불균형문제개선 .- StackedModel 앙상블기법을통해F1-score최대40%향상 .- Precision, ROC-AUC, F1-Score등다양한성능지표를활용하여모델유효성평가 .


### Self-Querying Retreiver

In [1]:
# %pip install --upgrade --quiet lark

In [ ]:
from langchain.schema import Document
from langchain_community.vectorstores import Chroma

docs = [
    Document(page_content="성격: 적극적이고 리더십이 뛰어나며 팀워크를 중요시하는 후보자. 여러 프로젝트에서 리더 역할을 맡았으며, 목표 달성을 위한 헌신적인 노력으로 성과를 이끌어낸 경험이 있습니다.",
             metadata={"experience": "5년간 프로젝트 관리 경험", "skills": "Leadership, Teamwork, Project Management", "education": "경영학 석사", "year": 1993}),
    Document(page_content="성격: 분석적이고 꼼꼼한 성격을 가진 후보자. 데이터 분석과 문제 해결 능력이 뛰어나며, 팀과 협력하여 복잡한 문제를 해결한 경험이 많습니다.",
             metadata={"experience": "3년간 데이터 분석 경험", "skills": "Data Analysis, Problem Solving, Collaboration", "education": "컴퓨터 공학 학사", "year": 1995}),
    Document(page_content="성격: 창의적이고 혁신적인 아이디어를 제시하는 것을 좋아하는 후보자. 새로운 기술과 트렌드를 빠르게 학습하고 프로젝트에 적용하는 데 강점을 보입니다.",
             metadata={"experience": "4년간 제품 개발 경험", "skills": "Creativity, Innovation, Rapid Learning", "education": "전자공학 석사", "year": 1998}),
    Document(page_content="성격: 친화력이 뛰어나며, 고객과의 소통을 중시하는 후보자. 고객 만족도를 높이기 위한 전략을 수립하고 실행한 경험이 있습니다.",
             metadata={"experience": "5년간 고객 서비스 관리 경험", "skills": "Customer Relations, Communication, Strategy", "education": "경영학 학사", "year": 2000}),
    Document(page_content="성격: 꼼꼼하고 정확한 성격을 가진 후보자. 다양한 프로젝트에서 기술적 문제를 해결하고 효율성을 극대화한 경험이 있습니다.",
             metadata={"experience": "6년간 엔지니어링 경험", "skills": "Attention to Detail, Efficiency, Technical Problem Solving", "education": "기계공학 석사", "year": 2002}),
    Document(page_content="성격: 유연하고 적응력이 뛰어난 후보자. 빠르게 변화하는 환경에서의 문제 해결 경험이 많으며, 새로운 상황에 맞는 최적의 솔루션을 제시합니다.",
             metadata={"experience": "4년간 IT 프로젝트 경험", "skills": "Adaptability, Problem Solving, Innovation", "education": "정보기술학 학사", "year": 2001}),
    Document(page_content="성격: 도전적이고 목표 지향적인 성격을 가진 후보자. 어려운 과제를 맡아 성과를 이끌어내며, 강한 동기부여를 가지고 일하는 특징이 있습니다.",
             metadata={"experience": "5년간 마케팅 전략 경험", "skills": "Challenge, Goal-Oriented, Marketing Strategy", "education": "마케팅 학사", "year": 1997}),
    Document(page_content="성격: 팀워크를 중시하고 타인의 의견을 존중하는 후보자. 협업을 통해 성과를 도출해낸 경험이 많으며, 팀 내에서 중요한 역할을 맡고 있습니다.",
             metadata={"experience": "3년간 프로젝트 협업 경험", "skills": "Teamwork, Collaboration, Leadership", "education": "사회학 석사", "year": 1994}),
    Document(page_content="성격: 분석적이며, 데이터를 바탕으로 의사결정을 내리는 것을 선호하는 후보자. 문제의 본질을 파악하고, 체계적인 방법으로 해결책을 제시합니다.",
             metadata={"experience": "7년간 데이터 분석 및 연구 경험", "skills": "Analytical Thinking, Data-Driven Decision Making, Research", "education": "통계학 석사", "year": 1999}),
    Document(page_content="성격: 끈기 있고, 목표를 달성하기 위해 노력하는 성격을 가진 후보자. 어떤 어려움도 이겨내며 주어진 목표를 달성해낸 경험이 있습니다.",
             metadata={"experience": "6년간 프로젝트 매니지먼트 경험", "skills": "Persistence, Goal Achievement, Project Management", "education": "토목공학 학사", "year": 2003}),
]

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli" # 임베딩 이용
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceEmbeddings(model_name=model_name,
                                   encode_kwargs=encode_kwargs)

vectorstore=Chroma.from_documents(docs, embeddings)

/tmp/ipykernel_3488/2490879139.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name,
2024-12-05 17:30:01.363033: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733387401.382973    3488 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733387401.389321    3488 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin c

In [4]:
import json

with open("api_keys.json") as f:
    OPENAI_API_KEY = json.load(f)["openai"]

In [5]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chat_models import ChatOpenAI

# 메타데이터 필드 정보 리스트 정의
metadata_field_info = [
    AttributeInfo(
        name = "experience",
        description = "The professional experience of the candidate",
        type = "string"
    ),
    AttributeInfo(
        name = "skills",
        description = "The skills possessed by the candidate",
        type = "string"
    ),
    AttributeInfo(
        name = "education",
        description = "The educational background of the candidate",
        type = "string"
    ),
    AttributeInfo(
        name = "year",
        description = "The year of the candidate's birth",
        type = "integer"
    ),
]

In [11]:
# LLM 모델 초기화
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# SelfQueryRetriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    metadata_field_info=metadata_field_info,
    document_contents="Candidate profile description",
    doc_content_description="A description of the candidate, including personality and experience.",
    verbose=True
)

In [40]:
query = "4년 이하의 경력을 가진 빠른 성장을 하는 후보"
results = retriever.get_relevant_documents(query)
for doc in results:
    print(doc.page_content)
    for metak, metav in doc.metadata.items():
        print(f"{metak}:{metav}")
    print("-"*55)
    

성격: 창의적이고 혁신적인 아이디어를 제시하는 것을 좋아하는 후보자. 새로운 기술과 트렌드를 빠르게 학습하고 프로젝트에 적용하는 데 강점을 보입니다.
education:전자공학 석사
experience:4년간 제품 개발 경험
skills:Creativity, Innovation, Rapid Learning
year:1998
-------------------------------------------------------
성격: 유연하고 적응력이 뛰어난 후보자. 빠르게 변화하는 환경에서의 문제 해결 경험이 많으며, 새로운 상황에 맞는 최적의 솔루션을 제시합니다.
education:정보기술학 학사
experience:4년간 IT 프로젝트 경험
skills:Adaptability, Problem Solving, Innovation
year:2001
-------------------------------------------------------
성격: 적극적이고 리더십이 뛰어나며 팀워크를 중요시하는 후보자. 여러 프로젝트에서 리더 역할을 맡았으며, 목표 달성을 위한 헌신적인 노력으로 성과를 이끌어낸 경험이 있습니다.
education:경영학 석사
experience:5년간 프로젝트 관리 경험
skills:Leadership, Teamwork, Project Management
year:1993
-------------------------------------------------------
성격: 꼼꼼하고 정확한 성격을 가진 후보자. 다양한 프로젝트에서 기술적 문제를 해결하고 효율성을 극대화한 경험이 있습니다.
education:기계공학 석사
experience:6년간 엔지니어링 경험
skills:Attention to Detail, Efficiency, Technical Problem Solving
year:2002
-------------------------------------------------------
